In [1]:
import time
import numpy as np
import pandas as pd
from scipy.io import mmread
from sklearn.metrics import silhouette_score
import matplotlib
import csv
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering
from sklearn.mixture import GaussianMixture
%matplotlib inline

In [9]:
data_DF = pd.read_csv("Reduced_data/diffusion_map_20_eigen.tsv", sep='\t',header=None)
data_DF = np.asarray(data_DF)

data_TSNE = pd.read_csv("Reduced_data/tsne_3.tsv", sep='\t',header=None)
data_TSNE = np.asarray(data_TSNE)

data_SVD = pd.read_csv("Reduced_data/truncatedsvd_20.tsv", sep='\t',header=None)
data_SVD = np.asarray(data_SVD)

data_PCA = pd.read_csv("Reduced_data/pca_20.csv",header=None)
data_PCA = np.asarray(data_PCA)

# Kmeans 

In [10]:
# Kmeans 
# TSNE all, SVD all

def calcSqDistances(X, Kmus):
    K = np.shape(Kmus)[0]
    N = np.shape(X)[0]
    D = np.shape(X)[1]
    
    # (N,K,D) - (K,D) = (N,K,D)
    # norm D--> (N,K)
    X_tile = np.tile(X, K)
    X_tile = X_tile.reshape(N,K,D)
    sqDmat = np.linalg.norm(X_tile-Kmus, axis=-1)
        
    return sqDmat

def determineRnk(sqDmat):
    # Rnk has shape (N,K)
    N = np.shape(sqDmat)[0]
    K = np.shape(sqDmat)[1]
    Rnk = np.zeros((N,K))
    min_idx = np.argmin(sqDmat, axis = 1)
    Rnk[np.arange(N),min_idx] = 1
    
    return Rnk, min_idx

def recalcMus(X, Rnk):
    K = np.shape(Rnk)[1]
    Kmus = Rnk.T @ X
    vecsum = np.sum(Rnk, axis=0)
    
    Kmus = Kmus/vecsum.reshape(len(vecsum),1) 
    return Kmus

def runKMeans(K, X):

    # Determine and store data set information
    N = np.shape(X)[0]
    D = np.shape(X)[1]

    # Allocate space for the K mu vectors
    Kmus = np.zeros((K, D))

    # Initialize cluster centers by randomly picking points from the data
    rndinds = np.random.permutation(N) # the maximum number of cluster is N.
    Kmus = X[rndinds[:K]];

    # Specify the maximum number of iterations to allow
    maxiters = 1000;
    
    start = time.time()

    for iter in range(maxiters):
        # Assign each data vector to closest mu vector as per Bishop (9.2)
        # Do this by first calculating a squared distance matrix where the n,k entry
        # contains the squared distance from the nth data vector to the kth mu vector

        # sqDmat will be an N-by-K matrix with the n,k entry as specfied above
        sqDmat = calcSqDistances(X, Kmus);
        
        # given the matrix of squared distances, determine the closest cluster
        # center for each data vector 

        # R is the "responsibility" matrix
        # R will be an N-by-K matrix of binary values whose n,k entry is set as 
        # per Bishop (9.2)
        # Specifically, the n,k entry is 1 if point n is closest to cluster k,
        # and is 0 otherwise
        Rnk, min_idx = determineRnk(sqDmat)
        
        KmusOld = Kmus

        # Recalculate mu values based on cluster assignments as per Bishop (9.4)
        Kmus = recalcMus(X, Rnk)

        # Check to see if the cluster centers have converged.  If so, break.
        if sum(abs(KmusOld.flatten() - Kmus.flatten())) < 1e-6:
            break
            
    end = time.time()
    runtime = end-start
    
    
    return runtime, min_idx

In [11]:
runtime_all = []
score_all = []
for K in [5,15,25]:
    time_trial = []
    score_trial = []
    fig, axes = plt.subplots(nrows=1, ncols=3)
    fig.set_size_inches(18.5, 5.5)
    plt.close()
    
    fig.suptitle('K-means clustering for K =' + str(K))
    count = 0
    while count<3:
        runtime, y_pred = runKMeans(K, data_DF)
        score = silhouette_score(data_DF, y_pred)
        axes[count].scatter(data_DF[:, 0], data_DF[:, 1], c=y_pred)
        axes[count].set_xlabel('Diffusion map 1')
        axes[count].set_ylabel('Diffusion map 2')
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
    name = 'DF_K-means'+str(K)+'.jpg'
    fig.savefig(name)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)

In [12]:
np.savetxt("DF_kmeans_silhouette.csv", score_all, delimiter=",") 
np.savetxt("DF_kmeans_runtime.csv", runtime_all, delimiter=",") 

In [ ]:
runtime_all = []
score_all = []
for K in [5,15,25]:
    time_trial = []
    score_trial = []
    fig, axes = plt.subplots(nrows=1, ncols=3)
    fig.set_size_inches(18.5, 5.5)
    plt.close()
    
    fig.suptitle('K-means clustering for K =' + str(K))
    count = 0
    while count<3:
        runtime, y_pred = runKMeans(K, data_TSNE)
        score = silhouette_score(data_TSNE, y_pred)
        axes[count].scatter(data_TSNE[:, 0], data_TSNE[:, 1], c=y_pred)
        axes[count].set_xlabel('TSNE 1')
        axes[count].set_ylabel('TSNE 2')
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
    name = 'TSNE_K-means'+str(K)+'.jpg'
    fig.savefig(name)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)

In [ ]:
np.savetxt("TSNE_kmeans_silhouette.csv", score_all, delimiter=",") 
np.savetxt("TSNE_kmeans_runtime.csv", runtime_all, delimiter=",") 

In [ ]:
runtime_all = []
score_all = []
y_pred_all = np.zeros(data_TSNE.shape[0]).reshape(data_TSNE.shape[0],1)
for K in [25]:
    time_trial = []
    score_trial = []

    count = 0
    while count<15:
        runtime, y_pred = runKMeans(K, data_TSNE)
        score = silhouette_score(data_TSNE, y_pred)
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
        y_pred_all = np.concatenate((y_pred_all, y_pred.reshape(data_TSNE.shape[0],1)),axis=1)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)
    y_pred_all = y_pred_all[:,1:16]
    y_pred_all = y_pred_all.astype(int)

In [ ]:
np.savetxt("TSNE_kmeans_ypred25.csv", y_pred_all, delimiter=",") 
np.savetxt("TSNE_kmeans_silhouette25.csv", score_all, delimiter=",") 
np.savetxt("TSNE_kmeans_runtime25.csv", runtime_all, delimiter=",") 

In [ ]:
runtime_all = []
score_all = []
for K in [5,15,25]:
    time_trial = []
    score_trial = []
    fig, axes = plt.subplots(nrows=1, ncols=3)
    fig.set_size_inches(18.5, 5.5)
    plt.close()
    
    fig.suptitle('K-means clustering for K =' + str(K))
    count = 0
    while count<3:
        runtime, y_pred = runKMeans(K, data_SVD)
        score = silhouette_score(data_SVD, y_pred)
        axes[count].scatter(data_SVD[:, 0], data_SVD[:, 1], c=y_pred)
        axes[count].set_xlabel('SVD 1')
        axes[count].set_ylabel('SVD 2')
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
    name = 'SVD_K-means'+str(K)+'.jpg'
    fig.savefig(name)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)

In [ ]:
np.savetxt("SVD_kmeans_silhouette.csv", score_all, delimiter=",") 
np.savetxt("SVD_kmeans_runtime.csv", runtime_all, delimiter=",") 

In [ ]:
runtime_all = []
score_all = []
y_pred_all = np.zeros(data_DF.shape[0]).reshape(data_SVD.shape[0],1)
for K in [25]:
    time_trial = []
    score_trial = []

    count = 0
    while count<15:
        runtime, y_pred = runKMeans(K, data_SVD)
        score = silhouette_score(data_SVD, y_pred)
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
        y_pred_all = np.concatenate((y_pred_all, y_pred.reshape(data_DF.shape[0],1)),axis=1)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)
    y_pred_all = y_pred_all[:,1:16]
    y_pred_all = y_pred_all.astype(int)

In [ ]:
np.savetxt("SVD_kmeans_ypred25.csv", y_pred_all, delimiter=",") 
np.savetxt("SVD_kmeans_silhouette25.csv", score_all, delimiter=",") 
np.savetxt("SVD_kmeans_runtime25.csv", runtime_all, delimiter=",") 

In [ ]:
runtime_all = []
score_all = []
for K in [5,15,25]:
    time_trial = []
    score_trial = []
    fig, axes = plt.subplots(nrows=1, ncols=3)
    fig.set_size_inches(18.5, 5.5)
    plt.close()
    
    fig.suptitle('K-means clustering for K =' + str(K))
    count = 0
    while count<3:
        runtime, y_pred = runKMeans(K, data_PCA)
        score = silhouette_score(data_PCA, y_pred)
        axes[count].scatter(data_PCA[:, 0], data_PCA[:, 1], c=y_pred)
        axes[count].set_xlabel('PCA 1')
        axes[count].set_ylabel('PCA 2')
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
    name = 'PCA_K-means'+str(K)+'.jpg'
    fig.savefig(name)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)

In [ ]:
np.savetxt("PCA_kmeans_silhouette.csv", score_all, delimiter=",") 
np.savetxt("PCA_kmeans_runtime.csv", runtime_all, delimiter=",") 

In [ ]:
runtime_all = []
score_all = []
y_pred_all = np.zeros(data_DF.shape[0]).reshape(data_PCA.shape[0],1)
for K in [25]:
    time_trial = []
    score_trial = []

    count = 0
    while count<15:
        runtime, y_pred = runKMeans(K, data_PCA)
        score = silhouette_score(data_PCA, y_pred)
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
        y_pred_all = np.concatenate((y_pred_all, y_pred.reshape(data_DF.shape[0],1)),axis=1)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)
    y_pred_all = y_pred_all[:,1:16]

In [ ]:
np.savetxt("PCA_kmeans_ypred25.csv", y_pred_all, delimiter=",") 
np.savetxt("PCA_kmeans_silhouette25.csv", score_all, delimiter=",") 
np.savetxt("PCA_kmeans_runtime25.csv", runtime_all, delimiter=",") 

## Gaussian

In [13]:
runtime_all = []
score_all = []
for K in [5,15,25]:
    time_trial = []
    score_trial = []
    fig, axes = plt.subplots(nrows=1, ncols=3)
    fig.set_size_inches(18.5, 5.5)
    plt.close()
    
    fig.suptitle('Gaussian mixture for K =' + str(K))
    count = 0
    while count<3:
        start = time.time()
        y_pred = GaussianMixture(n_components=K).fit_predict(data_DF)
        end = time.time()
        runtime = end-start
        
        score = silhouette_score(data_DF, y_pred)
        axes[count].scatter(data_DF[:, 0], data_DF[:, 1], c=y_pred)
        axes[count].set_xlabel('Diffusion map 1')
        axes[count].set_ylabel('Diffusion map 2')
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
    name = 'DF_gaussian'+str(K)+'.jpg'
    fig.savefig(name)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)

In [14]:
np.savetxt("DF_gaussian_silhouette.csv", score_all, delimiter=",") 
np.savetxt("DF_gaussian_runtime.csv", runtime_all, delimiter=",") 

In [ ]:
runtime_all = []
score_all = []
for K in [5,15,25]:
    time_trial = []
    score_trial = []
    fig, axes = plt.subplots(nrows=1, ncols=3)
    fig.set_size_inches(18.5, 5.5)
    plt.close()
    
    fig.suptitle('Gaussian mixture for K =' + str(K))
    count = 0
    while count<3:
        start = time.time()
        y_pred = GaussianMixture(n_components=K).fit_predict(data_TSNE)
        end = time.time()
        runtime = end-start
        
        score = silhouette_score(data_TSNE, y_pred)
        axes[count].scatter(data_TSNE[:, 0], data_TSNE[:, 1], c=y_pred)
        axes[count].set_xlabel('TSNE 1')
        axes[count].set_ylabel('TSNE 2')
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
    name = 'TSNE_gaussian'+str(K)+'.jpg'
    fig.savefig(name)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)

In [ ]:
np.savetxt("TSNE_gaussian_silhouette.csv", score_all, delimiter=",") 
np.savetxt("TSNE_gaussian_runtime.csv", runtime_all, delimiter=",") 

In [ ]:
runtime_all = []
score_all = []
y_pred_all = np.zeros(data_TSNE.shape[0]).reshape(data_TSNE.shape[0],1)
for K in [25]:
    time_trial = []
    score_trial = []

    count = 0
    while count<15:
        start = time.time()
        y_pred = GaussianMixture(n_components=K).fit_predict(data_TSNE)
        end = time.time()
        runtime = end-start
        
        score = silhouette_score(data_TSNE, y_pred)
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
        y_pred_all = np.concatenate((y_pred_all, y_pred.reshape(data_TSNE.shape[0],1)),axis=1)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)
    y_pred_all = y_pred_all[:,1:16].astype(int)

In [ ]:
np.savetxt("TSNE_gaussian_ypred25.csv", y_pred_all, delimiter=",") 
np.savetxt("TSNE_gaussian_silhouette25.csv", score_all, delimiter=",") 
np.savetxt("TSNE_gaussian_runtime25.csv", runtime_all, delimiter=",") 

In [ ]:
runtime_all = []
score_all = []
for K in [5,15,25]:
    time_trial = []
    score_trial = []
    fig, axes = plt.subplots(nrows=1, ncols=3)
    fig.set_size_inches(18.5, 5.5)
    plt.close()
    
    fig.suptitle('Gaussian mixture for K =' + str(K))
    count = 0
    while count<3:
        start = time.time()
        y_pred = GaussianMixture(n_components=K).fit_predict(data_SVD)
        end = time.time()
        runtime = end-start
        
        score = silhouette_score(data_SVD, y_pred)
        axes[count].scatter(data_SVD[:, 0], data_SVD[:, 1], c=y_pred)
        axes[count].set_xlabel('SVD 1')
        axes[count].set_ylabel('SVD 2')
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
    name = 'SVD_gaussian'+str(K)+'.jpg'
    fig.savefig(name)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)
    

In [ ]:
np.savetxt("SVD_gaussian_silhouette.csv", score_all, delimiter=",") 
np.savetxt("SVD_gaussian_runtime.csv", runtime_all, delimiter=",") 

In [ ]:
runtime_all = []
score_all = []
y_pred_all = np.zeros(data_DF.shape[0]).reshape(data_DF.shape[0],1)
for K in [25]:
    time_trial = []
    score_trial = []

    count = 0
    while count<15:
        start = time.time()
        y_pred = GaussianMixture(n_components=K).fit_predict(data_SVD)
        end = time.time()
        runtime = end-start
        
        score = silhouette_score(data_SVD, y_pred)
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
        y_pred_all = np.concatenate((y_pred_all, y_pred.reshape(data_PCA.shape[0],1)),axis=1)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)
    y_pred_all = y_pred_all[:,1:16].astype(int)

In [ ]:
np.savetxt("SVD_gaussian_ypred25.csv", y_pred_all, delimiter=",") 
np.savetxt("SVD_gaussian_silhouette25.csv", score_all, delimiter=",") 
np.savetxt("SVD_gaussian_runtime25.csv", runtime_all, delimiter=",") 

In [ ]:
runtime_all = []
score_all = []
for K in [5,15,25]:
    time_trial = []
    score_trial = []
    fig, axes = plt.subplots(nrows=1, ncols=3)
    fig.set_size_inches(18.5, 5.5)
    plt.close()
    
    fig.suptitle('Gaussian mixture for K =' + str(K))
    count = 0
    while count<3:
        start = time.time()
        y_pred = GaussianMixture(n_components=K).fit_predict(data_PCA)
        end = time.time()
        runtime = end-start
        
        score = silhouette_score(data_PCA, y_pred)
        axes[count].scatter(data_PCA[:, 0], data_PCA[:, 1], c=y_pred)
        axes[count].set_xlabel('PCA 1')
        axes[count].set_ylabel('PCA 2')
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
    name = 'PCA_gaussian'+str(K)+'.jpg'
    fig.savefig(name)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)

In [ ]:
np.savetxt("PCA_gaussian_silhouette.csv", score_all, delimiter=",") 
np.savetxt("PCA_gaussian_runtime.csv", runtime_all, delimiter=",") 

In [ ]:
runtime_all = []
score_all = []
y_pred_all = np.zeros(data_DF.shape[0]).reshape(data_DF.shape[0],1)
for K in [25]:
    time_trial = []
    score_trial = []

    count = 0
    while count<15:
        start = time.time()
        y_pred = GaussianMixture(n_components=K).fit_predict(data_PCA)
        end = time.time()
        runtime = end-start
        
        score = silhouette_score(data_PCA, y_pred)
        time_trial.append(runtime)
        score_trial.append(score)
        count += 1
        y_pred_all = np.concatenate((y_pred_all, y_pred.reshape(data_PCA.shape[0],1)),axis=1)
    runtime_K = np.mean(time_trial)
    runtime_all.append(runtime_K)
    score_K = np.mean(score_trial)
    score_all.append(score_K)
    y_pred_all = y_pred_all[:,1:16].astype(int)

In [ ]:
np.savetxt("PCA_gaussian_ypred25.csv", y_pred_all, delimiter=",") 
np.savetxt("PCA_gaussian_silhouette25.csv", score_all, delimiter=",") 
np.savetxt("PCA_gaussian_runtime25.csv", runtime_all, delimiter=",") 